In [1]:
from Datasets.data import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
import optuna.logging

optuna.logging.set_verbosity(optuna.logging.WARNING)

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='lightgbm')

/Users/davembiazi/Desktop/Projects/Model Correction/model_correction/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [4]:
from Models.model import train_lct, train_old_model, train_base_competitor_models,  train_competitor_standard_corrected
from Models.lct import LocalCorrectionTree

In [5]:
# Load and prepare data
def load_prep_data(dataset_name):
    if dataset_name in ['Adult', 'Bank']:
        dataset = Dataset(dataset_name)
        X, y, feature_names, x = dataset.load_data()
        X_old, y_old, X_new, y_new = dataset.split_old_new_data(X, y)
        return X_old, y_old, X_new, y_new, feature_names, x
    else:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

In [6]:
'''import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier as LGBM

dataset = Dataset("Bank")
X, y, feature_names, raw_data = dataset.load_data()
X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


old_model = LGBM(learning_rate=0.2, min_split_gain=0.3, reg_alpha=0.2, 
                 reg_lambda=0.2, class_weight='balanced')
old_model.fit(X_train, y_train)

# Get old model scores on the test set
old_scores = old_model.predict_proba(X_test)

# Instantiate and fit the Local Correction Tree
lct = LocalCorrectionTree()
lct.fit(X_test, y_test, old_scores)

#lct.prune(X_test, y_test, old_scores)
#lct.simplify()

# Get corrections from LCT
corrections = lct.predict(X_test)

# Apply corrections to old model scores
new_scores = old_scores + corrections

# Analyze the corrections
non_zero_corrections = np.sum(np.any(corrections != 0, axis=1))
print(f"Number of samples with non-zero corrections: {non_zero_corrections}")
print(f"Percentage of samples corrected: {non_zero_corrections / len(X_test) * 100:.2f}%")

 # Evaluate overall performance.
old_preds = np.argmax(old_scores, axis=1)
new_preds = np.argmax(new_scores, axis=1)
old_accuracy = np.mean(old_preds == y_test)
new_accuracy = np.mean(new_preds == y_test)
non_zero_corr = np.sum(np.linalg.norm(corrections, axis=1) > 1e-8)

print("Old model accuracy:", round(old_accuracy, 4))
print("Corrected model accuracy:", round(new_accuracy, 4))
print("Number of samples with non-zero corrections:", non_zero_corr)
print("Percentage of samples corrected: {:.2f}%".format(non_zero_corr / len(X_test) * 100))
print("\nExample corrections:")
for i in range(10):
    print("Sample", i, ":")
    print("  Old scores:", old_scores[i])
    print("  Correction:", corrections[i])
    print("  New scores:", new_scores[i])
    print("  True label:", y_test[i])
    print("  Predicted label before correction:", old_preds[i])
    print("  Predicted label after correction:", new_preds[i])
    print()

print("Simplified tree structure:")
lct.print_tree(feature_names=feature_names) '''

'import numpy as np\nfrom sklearn.datasets import make_classification\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LogisticRegression\nfrom lightgbm import LGBMClassifier as LGBM\n\ndataset = Dataset("Bank")\nX, y, feature_names, raw_data = dataset.load_data()\nX, y = np.array(X), np.array(y)\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n\nold_model = LGBM(learning_rate=0.2, min_split_gain=0.3, reg_alpha=0.2, \n                 reg_lambda=0.2, class_weight=\'balanced\')\nold_model.fit(X_train, y_train)\n\n# Get old model scores on the test set\nold_scores = old_model.predict_proba(X_test)\n\n# Instantiate and fit the Local Correction Tree\nlct = LocalCorrectionTree()\nlct.fit(X_test, y_test, old_scores)\n\n#lct.prune(X_test, y_test, old_scores)\n#lct.simplify()\n\n# Get corrections from LCT\ncorrections = lct.predict(X_test)\n\n# Apply corrections to old model scores\nnew_scores = old_scores + 

In [7]:
'''def _find_w(self, indices, old_scores, y, max_iter=10):
        #n_samples = len(indices)
        n_classes = old_scores.shape[1]
        
        w = np.zeros((1, n_classes))
        g_avg = np.zeros((1, n_classes))
        beta_t = 1.0  # Example sequence, adjust as needed
        
        for t in range(1, max_iter + 1):
            for i in indices:
                # Compute subgradient of the loss function for sample i
                sub_scores = old_scores[i] + w
                probs = np.exp(sub_scores) / np.sum(np.exp(sub_scores))
                subgradient = -np.eye(n_classes)[y[i]] + probs
                
                # Update average subgradient
                g_avg = (t - 1) / t * g_avg + 1 / t * subgradient
                
                # Update w using RDA formula
                w = w - beta_t / t * g_avg - self.lambda_reg * w
        
        return w'''

'def _find_w(self, indices, old_scores, y, max_iter=10):\n        #n_samples = len(indices)\n        n_classes = old_scores.shape[1]\n        \n        w = np.zeros((1, n_classes))\n        g_avg = np.zeros((1, n_classes))\n        beta_t = 1.0  # Example sequence, adjust as needed\n        \n        for t in range(1, max_iter + 1):\n            for i in indices:\n                # Compute subgradient of the loss function for sample i\n                sub_scores = old_scores[i] + w\n                probs = np.exp(sub_scores) / np.sum(np.exp(sub_scores))\n                subgradient = -np.eye(n_classes)[y[i]] + probs\n                \n                # Update average subgradient\n                g_avg = (t - 1) / t * g_avg + 1 / t * subgradient\n                \n                # Update w using RDA formula\n                w = w - beta_t / t * g_avg - self.lambda_reg * w\n        \n        return w'

In [8]:
###############################################################################
#   5. Example: Evaluate all methods with 5-fold cross validation
###############################################################################
def evaluate_all_methods(
    dataset_name, 
    X_new,
    X_old,
    y_new,
    y_old,
    localcorrectiontree_cls,
    random_state=42
):
    """
    Splits X,y into 5 folds. In each fold:
      1) Create a “train” (fold_train_idx) and “test” (fold_test_idx).
      2) Train or fine-tune models on the train set (with any hyperparameter
         tuning using that train set only).
      3) Evaluate on test.
    Accumulate test metrics for each method across folds. 
    Finally, produce boxplots for accuracy, precision, recall, F1. 
    """
    
    print("=== Training old model on X_old, y_old ===")
    old_model = train_old_model(dataset_name, X_old, y_old, model_type='LGBM')
    old_models_dict = {"LGBM": old_model}
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    
    # We’ll store results as: results[model_name]['accuracy'] = [fold1, fold2, ...]
    results = {}
    all_models = {}
    
    # (a) old models
    for old_m_name, old_m in old_models_dict.items():
        all_models[f"Old_{old_m_name}"] = old_m
        
    fold_idx = 1
    for train_index, test_index in skf.split(X_new, y_new):
        print(f"\n========= Fold {fold_idx} =========")
        fold_idx += 1
        
        X_train, X_test = X_new[train_index], X_new[test_index]
        y_train, y_test = y_new[train_index], y_new[test_index]
        
        # 2) Train the naive/new-only baselines
        base_models = train_base_competitor_models(dataset_name, X_train, y_train)
        
        # 3) Train standard correction methods
        competitor_corrected = train_competitor_standard_corrected(
            dataset_name, 
            X_train, 
            y_train,
            old_model=old_model,
            base_competitor_models=base_models
        )
        
        # 4) Train LCT to correct each old model
        lct_corrected = train_lct(
            dataset_name, 
            X_train, 
            y_train, 
            old_models_dict, 
            localcorrectiontree_cls
        )
        
        # 5) Evaluate each method on the test set. 
        #    This includes:
        #      - The old model(s)
        #      - The naive baselines (L1-LR, L2-LR, DT, RF, LGBM)
        #      - The standard corrections (L1-LR+, ..., LGBM+C, etc.)
        #      - Our LCT corrections
        
        # (b) naive/new-only
        for m_name, m in base_models.items():
            all_models[m_name] = m
        
        # (c) standard corrections
        for m_name, m in competitor_corrected.items():
            all_models[m_name] = m
        
        # (d) LCT
        #for m_name, m in lct_corrected.items():
        #    all_models[m_name] = m
        
        # Now actually do test predictions
        for model_key, model_obj in all_models.items():
            
            # If model_obj is an LCT, we do old_scores + correction, then argmax
            if isinstance(model_obj, LocalCorrectionTree):
                
                try:
                    old_probs = old_models_dict[model_key].predict_proba(X_test)
                except:
                    preds_ = old_models_dict[model_key].predict(X_test)
                    unique_labels = np.unique(preds_)
                    n_cls = len(unique_labels)
                    old_probs = np.zeros((len(X_test), n_cls))
                    for i, lab in enumerate(unique_labels):
                        old_probs[preds_ == lab, i] = 1.0
                        
                corrections_ = model_obj.predict(X_test)  # shape matching old_probs
                corrected_scores = old_probs + corrections_
                y_pred = np.argmax(corrected_scores, axis=1)
                
            else:
                # Normal scikit-learn model
                y_pred = model_obj.predict(X_test)
            
            # Evaluate metrics
            acc = accuracy_score(y_test, y_pred)
            # For binary or multi-class, set average appropriately
            if dataset_name in ['CTG', '7-point']:
                # multi-class => macro avg
                prec = precision_score(y_test, y_pred, average='macro')
                rec = recall_score(y_test, y_pred, average='macro')
                f1 = f1_score(y_test, y_pred, average='macro')
            else:
                # binary => standard
                prec = precision_score(y_test, y_pred)
                rec = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
            
            # Store
            if model_key not in results:
                results[model_key] = {
                    'accuracy': [],
                    'precision': [],
                    'recall': [],
                    'f1': []
                }
            results[model_key]['accuracy'].append(acc)
            results[model_key]['precision'].append(prec)
            results[model_key]['recall'].append(rec)
            results[model_key]['f1'].append(f1)
    
    # Once done with all folds, produce boxplots for each metric
    _plot_results_boxplots(results, dataset_name)
    
    return results

def _plot_results_boxplots(results_dict, dataset_name):
    """
    Create side-by-side horizontal bar plots (Accuracy, Precision, Recall, F1).
    Each bar is the AVERAGE score across all folds for the given model.
    The bars are arranged so that base models appear next to their '+'
    counterparts, e.g. 'L1-LR' next to 'L1-LR+'.
    """

    # Convert results to DataFrame
    rows = []
    for model_name, metrics in results_dict.items():
        n_folds = len(metrics['accuracy'])
        for i in range(n_folds):
            rows.append({
                'models':     model_name,
                'accuracy':  metrics['accuracy'][i],
                'precision': metrics['precision'][i],
                'recall':    metrics['recall'][i],
                'f1':        metrics['f1'][i]
            })
            
    df = pd.DataFrame(rows)
    
    # 2) Compute the AVERAGE across folds for each model
    df_mean = df.groupby('models', as_index=False).mean(numeric_only=True)

    desired_order = [
        "L1-LR", "L1-LR+",
        "L2-LR", "L2-LR+",
        "DT", "DT+",
        "RF", "RF+",
        "LGBM", "LGBM+", "LGBM+C",
        "Ours", "Old_LGBM"
    ]
    
    # Keep only those that actually appear
    model_order = [m for m in desired_order if m in df_mean['models'].unique()]

    # Set up a whitegrid style and a smaller figure
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=0.8)
    
    # Adjust the figure size to make it smaller
    fig, axes = plt.subplots(1, 4, figsize=(8, 4), sharey=True)

    # Helper to plot one metric as a bar chart
    def plot_metric(ax, metric_col, title_str):
        # Subset & reorder so the bars appear in the desired order
        plot_data = df_mean.set_index('models').loc[model_order].reset_index()
        
        sns.barplot(
            data=plot_data,
            x=metric_col,       # numeric metric on the x-axis
            y='models',          # model on the y-axis
            ax=ax, 
            orient='h',         # horizontal bars
            palette='Set2',     # different color per model
        )
        ax.set_title(f"{title_str} ({dataset_name})")
        ax.set_xlim([0.0, 1.0])  # Adjust as needed for your data range
        ax.legend()
        
    # Draw each subplot
    plot_metric(axes[0], 'accuracy',  'Accuracy')
    plot_metric(axes[1], 'precision', 'Precision')
    plot_metric(axes[2], 'recall',    'Recall')
    plot_metric(axes[3], 'f1',        'F1')
    
    #plt.title(f"Comparison plot for_{dataset_name}")
    plt.grid(True, axis='x', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(f"Comparison plot for_{dataset_name}.png", dpi=300)
    plt.show()

In [10]:
###############################################################################
#   USAGE EXAMPLE (Pseudocode)
###############################################################################
if __name__ == "__main__":
    
    X_old, y_old, X_new, y_new, feature_names, x = load_prep_data('Adult')    

    X_old, y_old = np.array(X_old), np.array(y_old)
    X_new, y_new = np.array(X_new), np.array(y_new)
    
    results = evaluate_all_methods(
        dataset_name='Adult',
        X_new=X_new[:100], 
        X_old=X_old[:100],
        y_new=y_new[:100],
        y_old=y_old[:100],
        localcorrectiontree_cls=LocalCorrectionTree
        )


KeyboardInterrupt: 